In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy

In [5]:
# Create a set of random latitude and longitude combinations.
lats =  np.random.uniform(low=-90.000, high= 90.000,size=1500)
lngs = np.random.uniform(low=-180.000, high= 180.000,size=1500)
lat_lngs = zip(lats,lngs)
lat_lngs


In [6]:
coordinates=list(lat_lngs)

In [7]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and logitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

572

In [8]:
# Import the datetime module from the datetime library
from datetime import datetime

In [9]:
# Import the requests library
import requests

# Import the API key.
from config import weather_api_key

# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
        

In [10]:
# Create and empty list to hold the weather data.
city_data=[]
# Print the beginning of the logging.
print("Beginning Data Retrieval      ")
print("------------------------------")

# Create counters.
record_count = 1
set_count = 1

Beginning Data Retrieval      
------------------------------


In [11]:
# Loop through all the cities in our list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    if(i % 50 == 0 and i >= 50):
        set_count += 1
        record_count += 1
    # Create endpoint url with each city.
    city_url=url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record{record_count} of Set {set_count} | {city}")
    record_count += 1
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Processing Record1 of Set 1 | amderma
City not found. Skipping...
Processing Record2 of Set 1 | taolanaro
City not found. Skipping...
Processing Record3 of Set 1 | mataura
Processing Record4 of Set 1 | avarua
Processing Record5 of Set 1 | bredasdorp
Processing Record6 of Set 1 | nikolskoye
Processing Record7 of Set 1 | lakhnadon
Processing Record8 of Set 1 | luderitz
Processing Record9 of Set 1 | ushuaia
Processing Record10 of Set 1 | hasaki
Processing Record11 of Set 1 | kudahuvadhoo
Processing Record12 of Set 1 | bethel
Processing Record13 of Set 1 | dien bien
City not found. Skipping...
Processing Record14 of Set 1 | bluff
Processing Record15 of Set 1 | belushya guba
City not found. Skipping...
Processing Record16 of Set 1 | qaanaaq
Processing Record17 of Set 1 | endicott
Processing Record18 of Set 1 | punta arenas
Processing Record19 of Set 1 | dikson
Processing Record20 of Set 1 | puerto escondido
Processing Record21 of Set 1 | chokurdakh
Processing Record22 of Set 1 | rikitea
Pro

Processing Record191 of Set 4 | mattru
Processing Record192 of Set 4 | rio grande
Processing Record193 of Set 4 | marawi
Processing Record194 of Set 4 | khonuu
City not found. Skipping...
Processing Record195 of Set 4 | fortuna
Processing Record196 of Set 4 | macapa
Processing Record197 of Set 4 | bubaque
Processing Record198 of Set 4 | tuatapere
Processing Record199 of Set 4 | manakara
Processing Record200 of Set 4 | kerouane
Processing Record201 of Set 4 | asayita
City not found. Skipping...
Processing Record202 of Set 4 | lasa
Processing Record203 of Set 4 | geilo
Processing Record205 of Set 5 | khartoum
Processing Record206 of Set 5 | bengkulu
Processing Record207 of Set 5 | porto novo
Processing Record208 of Set 5 | businga
Processing Record209 of Set 5 | novokayakent
Processing Record210 of Set 5 | salina
Processing Record211 of Set 5 | kingswood
Processing Record212 of Set 5 | cayenne
Processing Record213 of Set 5 | kamaishi
Processing Record214 of Set 5 | araouane
Processing Re

Processing Record388 of Set 8 | yandoon
City not found. Skipping...
Processing Record389 of Set 8 | bonoua
Processing Record390 of Set 8 | porto walter
Processing Record391 of Set 8 | pundaguitan
Processing Record392 of Set 8 | umkomaas
Processing Record393 of Set 8 | kiama
Processing Record394 of Set 8 | foam lake
Processing Record395 of Set 8 | attawapiskat
City not found. Skipping...
Processing Record396 of Set 8 | bandarbeyla
Processing Record397 of Set 8 | mutis
Processing Record398 of Set 8 | lorengau
Processing Record399 of Set 8 | harlingen
Processing Record400 of Set 8 | qaqortoq
Processing Record401 of Set 8 | aksarka
Processing Record402 of Set 8 | kazanskoye
Processing Record403 of Set 8 | gat
Processing Record404 of Set 8 | berlevag
Processing Record405 of Set 8 | makat
Processing Record406 of Set 8 | lachi
Processing Record407 of Set 8 | orgun
City not found. Skipping...
Processing Record409 of Set 9 | santa maria del oro
Processing Record410 of Set 9 | impfondo
Processin

Processing Record576 of Set 12 | kukushtan
Processing Record577 of Set 12 | zigong
Processing Record578 of Set 12 | acarau
Processing Record579 of Set 12 | warqla
City not found. Skipping...
Processing Record580 of Set 12 | gusau
Processing Record581 of Set 12 | ivaipora
Processing Record582 of Set 12 | caruray
Processing Record583 of Set 12 | yiyang
-----------------------------
Data Retrieval Complete      
-----------------------------


In [13]:
print(len(city_data))

521


In [ ]:
# Convert the array of dictionaryies to a Pandas DataFrame.
city_data_df= pd.DataFrame(city_data)
city_data_df.head(10)

In [ ]:
new_order=["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]
city_data_df=city_data_df.reindex(columns=new_order)
city_data_df


In [ ]:
# Create teh output file (CSV).
output_data_file = "weather_data_cities.csv"
# Export the City_data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

In [ ]:
# Extract relevant fields from the DataFrame for plotting.
lats = city_data_df["Lat"]
max_temps = city_data_df["Max Temp"]
humidity = city_data_df["Humidity"]
cloudiness = city_data_df["Cloudiness"]
wind_speed = city_data_df["Wind Speed"]


In [ ]:
# Import time module
import time
# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,max_temps, edgecolor="black",linewidth=1,marker="o",alpha = .8,label="Cities")

# Incorporate the other graph properties
plt.title(f"City Latitude vs. Max Temperature "+ time.strftime("%x"))
plt.ylabel("Max Temperature (F)")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
plt.savefig("weather_data-Fig1.png")

# Show plot
plt.show()

In [ ]:
# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,humidity, edgecolor="black",linewidth=1,marker="o",alpha = .8,label="Cities")

# Incorporate the other graph properties
plt.title(f"City Latitude vs. Humidity "+ time.strftime("%x"))
plt.ylabel("Humidity as %")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
plt.savefig("weather_data-Fig2-Humidity.png")

# Show plot
plt.show()

In [ ]:
# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,cloudiness, edgecolor="black",linewidth=1,marker="o",alpha = .8,label="Cities")

# Incorporate the other graph properties
plt.title(f"City Latitude vs.Cloudiness "+ time.strftime("%x"))
plt.ylabel("Cloudiness as %")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
plt.savefig("weather_data-Fig3-Cloudiness.png")

# Show plot
plt.show()

In [ ]:
# Build the scatter plot for latitude vs. max temperature.
plt.scatter(lats,wind_speed, edgecolor="black",linewidth=1,marker="o",alpha = .8,label="Cities")

# Incorporate the other graph properties
plt.title(f"City Latitude vs. Wind Speed "+ time.strftime("%x"))
plt.ylabel("Wind Speed")
plt.xlabel("Latitude")
plt.grid(True)

# Save the figure.
plt.savefig("weather_data-Fig4-Wind Speed.png")

# Show plot
plt.show()

In [ ]:
# Import linregress
from scipy.stats import linregress

In [ ]:
# Create a function to create perform linear regression on the weather data
# and plot a regression line and the equation with the data.
def plot_linear_regression(x_values, y_values, title, y_label, text_coordinates):
    
    # Run regression on hemisphere weather data.
    (slope,intercept,r_value,p_value,std_err) = linregress(x_values,y_values)
    
    # Calculate the regression line "y values" from the slope and intercept.
    regress_values = x_values * slope + intercept
    
    # Get the equation of the line.
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept))
    
    # Get the r value
    print(f"The r-value is {r_value:.3f}")
    
    # Create a scatter plot and plot the regression line.
    plt.scatter(x_values,y_values)
    plt.plot(x_values,regress_values,"r")
    # Annotate the text for the line equation.
    plt.annotate(line_eq, text_coordinates, fontsize=15, color="red")
    plt.title(title)
    plt.xlabel('Latitude')
    plt.ylabel(y_label)
    plt.show()

In [ ]:
index13 = city_data_df.loc[13]
index13

In [ ]:
city_data_df["Lat"]>=0

In [ ]:
city_data_df.loc[(city_data_df["Lat"]>=0)].head()

In [ ]:
# Create Northern and Southern Hemisphere DataFrames.
northern_hemi_df = city_data_df.loc[(city_data_df["Lat"]>=0)]
southern_hemi_df = city_data_df.loc[(city_data_df["Lat"]<0)]

In [ ]:
# Linear regression on the Northern Hemisphere
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Max Temp"]

# Call the function
plot_linear_regression(x_values,y_values,"Linear Regression on the Northern Hemisphere \ for Maximum Temperature","Max Temp",(10,40))

In [ ]:
# Linear regression on the Southern Hemisphere
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Max Temp"]

# Call the function
plot_linear_regression(x_values,y_values,"Linear Regression on the Southern Hemisphere \ for Maximum Temperature","Max Temp",(-20,45))


In [ ]:
# Linear regression on the Northern Hemisphere - Humidity
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Humidity"]

# Call the function
plot_linear_regression(x_values,y_values,"Linear Regression on the Northern Hemisphere \ for Humidity %","Humidity",(10,40))

In [ ]:
# Linear regression on the Northern Hemisphere - Humidity
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Humidity"]

# Call the function
plot_linear_regression(x_values,y_values,"Linear Regression on the Southern Hemisphere \ for Humidity %","Humidity",(-50,20))

In [ ]:
# Linear regression on the Northern Hemisphere - Cloudiness
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Cloudiness"]

# Call the function
plot_linear_regression(x_values,y_values,"Linear Regression on the Northern Hemisphere \ for % Cloudiness","Cloudiness",(10,40))

In [ ]:
# Linear regression on the Southern Hemisphere - Cloudiness
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Cloudiness"]

# Call the function
plot_linear_regression(x_values,y_values,"Linear Regression on the Southern Hemisphere \ for % Cloudiness","Cloudiness",(-50,40))

In [ ]:
# Linear regression on the Northern Hemisphere - Wind Speed
x_values = northern_hemi_df["Lat"]
y_values = northern_hemi_df["Wind Speed"]

# Call the function
plot_linear_regression(x_values,y_values,"Linear Regression on the Northern Hemisphere \ for % Wind Speed","Wind Speed",(10,30))

In [ ]:
# Linear regression on the southern Hemisphere - Wind Speed
x_values = southern_hemi_df["Lat"]
y_values = southern_hemi_df["Wind Speed"]

# Call the function
plot_linear_regression(x_values,y_values,"Linear Regression on the Southern Hemisphere \ for % Wind Speed","Wind Speed",(-50,25))